In [1]:
file_location = "/FileStore/tables/purchases.txt"
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "false").option("inferSchema", "true").option("delimiter", "\t").load(file_location)

In [2]:
new_df = spark.read.options(inferSchema = 'True', delimiter = '\t', header = 'False')\
          .csv(file_location)
new_df.show(3)

+----------+-----+----------+----------------+------+----+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+-----+----------+----------------+------+----+
2012-01-01|09:00| San Jose| Men's Clothing|214.05|Amex|
2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|Visa|
2012-01-01|09:00| San Diego| Music| 66.08|Cash|
+----------+-----+----------+----------------+------+----+
only showing top 3 rows

In [3]:
df.show(3,truncate=True)

+----------+-----+----------+----------------+------+----+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+-----+----------+----------------+------+----+
2012-01-01|09:00| San Jose| Men's Clothing|214.05|Amex|
2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|Visa|
2012-01-01|09:00| San Diego| Music| 66.08|Cash|
+----------+-----+----------+----------------+------+----+
only showing top 3 rows

In [4]:
#rename columns
old_columns = df.schema.names
new_columns = ["Date","Time","Store_Name","Item","Sale","Payment_Type"]
for i in range(len(old_columns)):
    df = df.withColumnRenamed(old_columns[i], new_columns[i])
df.show(3)

+----------+-----+----------+----------------+------+------------+
 Date| Time|Store_Name| Item| Sale|Payment_Type|
+----------+-----+----------+----------------+------+------------+
2012-01-01|09:00| San Jose| Men's Clothing|214.05| Amex|
2012-01-01|09:00|Fort Worth|Women's Clothing|153.57| Visa|
2012-01-01|09:00| San Diego| Music| 66.08| Cash|
+----------+-----+----------+----------------+------+------------+
only showing top 3 rows

In [5]:
rdd = df.rdd
rdd.take(2)

Out[7]: [Row(Date='2012-01-01', Time='09:00', Store_Name='San Jose', Item="Men's Clothing", Sale=214.05, Payment_Type='Amex'),
 Row(Date='2012-01-01', Time='09:00', Store_Name='Fort Worth', Item="Women's Clothing", Sale=153.57, Payment_Type='Visa')]

In [6]:
df.filter(df.Store_Name == "San Jose").select(df.Store_Name, df.Payment_Type).show(10)

+----------+------------+
Store_Name|Payment_Type|
+----------+------------+
 San Jose| Amex|
 San Jose| Cash|
 San Jose| Cash|
 San Jose| Cash|
 San Jose| Cash|
 San Jose| Discover|
 San Jose| Amex|
 San Jose| MasterCard|
 San Jose| Cash|
 San Jose| Visa|
+----------+------------+
only showing top 10 rows

In [7]:
from pyspark.sql.functions import collect_set
distinct_df = df.where(df.Store_Name == "San Jose").select(df.Store_Name,df.Payment_Type).distinct()
distinct_df.groupBy('Store_Name').agg(collect_set('Payment_Type').alias('Types')).show(truncate=False)

+----------+----------------------------------------+
Store_Name|Types |
+----------+----------------------------------------+
San Jose |[Visa, Discover, Amex, Cash, MasterCard]|
+----------+----------------------------------------+

In [8]:
#displaying average sales by Store Name
from pyspark.sql.functions import format_number #used to display number in specific format
df.groupBy('Store_Name').agg({'Sale': 'avg'}).withColumn('avg(Sale)',format_number('avg(Sale)',2)).show(10)

+---------------+---------+
 Store_Name|avg(Sale)|
+---------------+---------+
North Las Vegas| 250.66|
 Phoenix| 249.90|
 Omaha| 249.36|
 Anchorage| 249.55|
 Anaheim| 251.37|
 Greensboro| 249.40|
 Dallas| 249.37|
 Oakland| 250.38|
 Laredo| 251.47|
 Scottsdale| 249.87|
+---------------+---------+
only showing top 10 rows

In [9]:
#displaying average sales by Store Name by date
df.groupBy('Store_Name','Date').agg({'Sale': 'avg'}).withColumnRenamed('avg(Sale)','average_sale').show(10)

+-------------+----------+------------------+
 Store_Name| Date| average_sale|
+-------------+----------+------------------+
 Spokane|2012-01-02| 247.2948484848485|
 Chula Vista|2012-01-03|251.65900990099001|
 Milwaukee|2012-01-04|255.48515463917522|
Oklahoma City|2012-01-04|255.12386792452838|
 Garland|2012-01-07|252.88191304347825|
 Fremont|2012-01-08| 250.1847552447552|
 Saint Paul|2012-01-08| 271.120198019802|
 Denver|2012-01-13|249.81301075268826|
 Bakersfield|2012-01-14| 238.17136|
 New Orleans|2012-01-15| 230.808547008547|
+-------------+----------+------------------+
only showing top 10 rows

In [10]:
#using SQL ......total sales by Store Name
df.createOrReplaceTempView("sales")
sqlContext.sql("Select Store_Name, sum(Sale) as total_sales from sales group by Store_Name").toPandas()[:5]

In [11]:
#average sales by store - note that I have used cast to display up to 2 decimal places
sqlContext.sql("Select Store_Name, cast(avg(Sale) as decimal(15,2)) as average_sales from sales group by Store_Name").toPandas()[:5]

Out[ 7 ]: 
 Store_Name average_sales
0 North Las Vegas 250.66
1 Phoenix 249.90
2 Omaha 249.36
3 Anchorage 249.55
4 Anaheim 251.37